In [53]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [54]:
FeatureSetB = pd.read_csv('MIMIC_Windows/FSB_1Ws.csv')
FeatureMortality = pd.read_csv('MIMIC_Windows/FSB_Mortality.csv')
numberOfWindows = 1
numberOfFeatures = 20

In [55]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)
Subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(Subject_id)

In [56]:
X = FeatureMortality['subject_id'].equals(Subject_id)

In [57]:
FeatureSet = FeatureSetB.drop(['subject_id'], axis=1)

In [58]:
FeatureSet = FeatureSet.to_numpy()

In [59]:
Labels = pd.DataFrame()
Labels['Expired'] = FeatureMortality['Expired'] 
y_values = Labels.to_numpy()
y_values = y_values.flatten()

In [61]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(FeatureSet)
X_scaled = scaler.transform(FeatureSet)

In [62]:
nClients = NumSubjects
featureArray = X_scaled.reshape(nClients,numberOfWindows*numberOfFeatures)

Testing SVM

In [63]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from collections import Counter

 
acc_score = []
re_score = []
pre_score = []
f_score = []

In [64]:
import warnings
warnings.filterwarnings("ignore")

In [65]:
from numpy.random import seed
seed(123)

import sys
mod=sys.modules[__name__]
from sklearn import svm
model = svm.SVC(kernel='poly')
from sklearn import metrics

acc_score = []
re_score = []
pre_score = []
f_score = []
auroc = []
auprc = []

from sklearn.model_selection import KFold 
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

X = featureArray
y = y_values
resample = NeighbourhoodCleaningRule()
k = 10

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

for train_ix, val_ix in skf.split(X,y):
    # select rows
    X_train, X_val = X[train_ix], X[val_ix]
    y_train, y_val = y[train_ix], y[val_ix]

    X_train,y_train = resample.fit_resample(X_train, y_train) 

    history4 = model.fit(X_train,y_train) 
    y_pred = model.predict(X_val)
    y_pred = list(map(lambda x: 0 if x<=0.5 else 1, y_pred))

    y_actu = pd.Series(y_val)
    y_pred = pd.Series(y_pred)
    
    f1_value = f1_score(y_actu, y_pred,pos_label = 1, average='binary') 
    sensitivity = recall_score(y_actu, y_pred,pos_label = 1, average='binary')
    precision = precision_score(y_actu, y_pred,pos_label = 1, average='binary')
    prc = average_precision_score(y_actu, y_pred,pos_label = 1)
    roc = roc_auc_score(y_actu, y_pred)
    accuracy = accuracy_score(y_actu, y_pred)
    
    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)
    auroc.append(roc)
    auprc.append(prc)
   
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_roc_score = sum(auroc)/k
avg_prc_score = sum(auprc)/k

sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score
auroc = avg_roc_score
auprc = avg_prc_score

In [66]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
print('Recall of each fold - {}'.format(re_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Precision of each fold - {}'.format(pre_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('F1_Score of each fold - {}'.format(f_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('AUROC of each fold - {}'.format(auroc))
print('Avg AUROC : {}'.format(avg_roc_score))
print('AUPRC of each fold - {}'.format(auprc))
print('Avg AUPRC : {}'.format(avg_prc_score))  

accuracy of each fold - [0.9108391608391608, 0.9090909090909091, 0.9073426573426573, 0.9003496503496503, 0.9003496503496503, 0.9055944055944056, 0.9073426573426573, 0.9090909090909091, 0.9020979020979021, 0.9036777583187391]
Avg accuracy : 0.9055775660416643
Recall of each fold - [0.43137254901960786, 0.49019607843137253, 0.5, 0.38461538461538464, 0.5384615384615384, 0.4423076923076923, 0.38461538461538464, 0.5384615384615384, 0.4807692307692308, 0.4117647058823529]
Avg Reccall : 0.46025641025641023
Precision of each fold - [0.5, 0.49019607843137253, 0.49056603773584906, 0.4444444444444444, 0.45901639344262296, 0.4791666666666667, 0.4878048780487805, 0.5, 0.46296296296296297, 0.45652173913043476]
Avg Precision : 0.4770679200863134
F1_Score of each fold - [0.4631578947368421, 0.49019607843137253, 0.49523809523809526, 0.4123711340206186, 0.4955752212389381, 0.45999999999999996, 0.4301075268817205, 0.5185185185185186, 0.4716981132075472, 0.4329896907216495]
Avg F1_score : 0.46698522729953

In [67]:
import csv
import os.path
from datetime import datetime

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','AUROC','AUPRC','NumberOfWindows']
dict_data = [{'model-type':'SVM', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'NumberOfWindows':numberOfWindows,'AUROC':auroc,'AUPRC':auprc}]
metric_file = "Results/Results_SVM.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

NameError: name 'ep' is not defined